In [1]:
import pandas as pd

In [2]:
# 1. Load data
url = 'https://raw.githubusercontent.com/XinyuanD/Datathon2026/refs/heads/main/Access_to_Care_Dataset.csv'
df = pd.read_csv(url)
df.head()

,TOPIC,SUBTOPIC,SUBTOPIC_ID,TAXONOMY,TAXONOMY_ID,CLASSIFICATION,CLASSIFICATION_ID,GROUP,GROUP_ID,GROUP_ORDER,...,ESTIMATE_TYPE,ESTIMATE_TYPE_ID,TIME_PERIOD,TIME_PERIOD_ID,ESTIMATE,STANDARD_ERROR,ESTIMATE_LCI,ESTIMATE_UCI,FLAG,FOOTNOTE_ID_LIST
0,Angina/angina pectoris,NaN,NaN,Cardiovascular diseases,60,Total,0,Total,1,1,...,"Percent of population, crude",1,2019,NaN,1.7,NaN,1.5,1.9,NaN,"NT_NHISA00,NT_NHISA999,FN_NHISA18,SC_NHISA00"
1,Angina/angina pectoris,NaN,NaN,Cardiovascular diseases,60,Total,0,Total,1,1,...,"Percent of population, crude",1,2020,NaN,1.5,NaN,1.3,1.6,NaN,"NT_NHISA00,NT_NHISA999,FN_NHISA18,SC_NHISA00"
2,Angina/angina pectoris,NaN,NaN,Cardiovascular diseases,60,Total,0,Total,1,1,...,"Percent of population, crude",1,2021,NaN,1.5,NaN,1.4,1.7,NaN,"NT_NHISA00,NT_NHISA999,FN_NHISA18,SC_NHISA00"
3,Angina/angina pectoris,NaN,NaN,Cardiovascular diseases,60,Total,0,Total,1,1,...,"Percent of population, crude",1,2022,NaN,1.6,NaN,1.5,1.8,NaN,"NT_NHISA00,NT_NHISA999,FN_NHISA18,SC_NHISA00"
4,Angina/angina pectoris,NaN,NaN,Cardiovascular diseases,60,Total,0,Total,1,1,...,"Percent of population, crude",1,2023,NaN,1.6,NaN,1.4,1.8,NaN,"NT_NHISA00,NT_NHISA999,FN_NHISA18,SC_NHISA00"


In [3]:
# 2. Drop "Metadata" and "Leakage" columns
# We remove the confidence intervals so the model doesn't "cheat"
redundant_ids = [col for col in df.columns if '_ID' in col]
leakage_cols = ['ESTIMATE_LCI', 'ESTIMATE_UCI', 'STANDARD_ERROR']
metadata_cols = ['FLAG', 'FOOTNOTE_ID_LIST', 'SUBTOPIC', 'SUBTOPIC_ID']
cols_to_drop = redundant_ids + leakage_cols + metadata_cols
df_clean = df.drop(columns=cols_to_drop, errors='ignore')

# 3. Handle the "Mostly NaN" columns automatically
# We set a threshold: if a column is more than 70% empty, we drop it
threshold = 0.3 * len(df)
df_clean = df_clean.dropna(thresh=threshold, axis=1)

# 4. Clean the Target Variable (ESTIMATE)
# You cannot train a model to predict something if the answer is missing!
df_clean = df_clean.dropna(subset=['ESTIMATE'])

# 5. Type Casting
# Ensure TIME_PERIOD is a number and IDs are strings (Categorical)
df_clean['TIME_PERIOD'] = pd.to_numeric(df_clean['TIME_PERIOD'], errors='coerce')
id_cols = [col for col in df_clean.columns if '_ID' in col]
df_clean[id_cols] = df_clean[id_cols].astype(str)

print(f"Original shape: {df.shape}")
print(f"Cleaned shape: {df_clean.shape}")
print("\nFinal Columns for Modeling:")
print(df_clean.columns.tolist())
df_clean.head()

Original shape: (26208, 25)
Cleaned shape: (23839, 10)

Final Columns for Modeling:
['TOPIC', 'TAXONOMY', 'CLASSIFICATION', 'GROUP', 'GROUP_ORDER', 'SUBGROUP', 'SUBGROUP_ORDER', 'ESTIMATE_TYPE', 'TIME_PERIOD', 'ESTIMATE']


,TOPIC,TAXONOMY,CLASSIFICATION,GROUP,GROUP_ORDER,SUBGROUP,SUBGROUP_ORDER,ESTIMATE_TYPE,TIME_PERIOD,ESTIMATE
0,Angina/angina pectoris,Cardiovascular diseases,Total,Total,1,18 years and older,1,"Percent of population, crude",2019,1.7
1,Angina/angina pectoris,Cardiovascular diseases,Total,Total,1,18 years and older,1,"Percent of population, crude",2020,1.5
2,Angina/angina pectoris,Cardiovascular diseases,Total,Total,1,18 years and older,1,"Percent of population, crude",2021,1.5
3,Angina/angina pectoris,Cardiovascular diseases,Total,Total,1,18 years and older,1,"Percent of population, crude",2022,1.6
4,Angina/angina pectoris,Cardiovascular diseases,Total,Total,1,18 years and older,1,"Percent of population, crude",2023,1.6


In [4]:
total_null_count = df_clean.isnull().sum().sum()

print("Total number of NaN or empty values:", total_null_count)

Total number of NaN or empty values: 0


In [ ]:
df_clean.to_csv('cleaned_health_data_xinyuan.csv', index=False)